In [3]:
# !nvidia-smi

In [4]:
!pip install icrawler

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from icrawler.builtin import BingImageCrawler
import shutil
import os

# 学習用画像の用意

In [7]:
# 猫の画像を100枚取得
crawler = BingImageCrawler(storage={"root_dir": "drive/MyDrive/Colab Notebooks/dataset/inuneko/train/cat"})
crawler.crawl(keyword="Cat", max_num=100)

2021-10-28 06:28:32,643 - INFO - icrawler.crawler - start crawling...
2021-10-28 06:28:32,646 - INFO - icrawler.crawler - starting 1 feeder threads...
2021-10-28 06:28:32,658 - INFO - feeder - thread feeder-001 exit
2021-10-28 06:28:32,658 - INFO - icrawler.crawler - starting 1 parser threads...
2021-10-28 06:28:32,674 - INFO - icrawler.crawler - starting 1 downloader threads...
2021-10-28 06:28:32,943 - INFO - parser - parsing result page https://www.bing.com/images/async?q=Cat&first=0
2021-10-28 06:28:34,481 - ERROR - downloader - Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_November_2010-1a.jpg
2021-10-28 06:28:34,550 - INFO - downloader - image #1	https://www.thesprucepets.com/thmb/A-lj23wOXsDRc_m3xNlwTl-tGPU=/6200x4138/filters:fill(auto,1)/GettyImages-523693627-976d1977942142c8b171a95057791a3e.jpg
2021-10-28 06:28:34,611 - INFO - downloader - image #2	https://pbs.twimg.com/media/DUfDNvTVQAA2osS.jpg
2021-10-28 06:28:34,637 - INFO - do

In [8]:
os.makedirs('drive/MyDrive/Colab Notebooks/dataset/inuneko/test/cat', exist_ok=True)

In [9]:
# 30枚を検証データに
for filenum in range(30):
  shutil.move('drive/MyDrive/Colab Notebooks/dataset/inuneko/train/cat/' +str(filenum+1).zfill(6)+'.jpg', 'drive/MyDrive/Colab Notebooks/dataset/inuneko/test/cat')

Error: ignored

In [ ]:
# 犬の画像を100枚取得
crawler = BingImageCrawler(storage={"root_dir": "drive/MyDrive/Colab Notebooks/dataset/inuneko/train/dog"})
crawler.crawl(keyword="Dog", max_num=100)

In [ ]:
os.makedirs('drive/MyDrive/Colab Notebooks/dataset/inuneko/test/dog', exist_ok=True)

In [ ]:
# 30枚を検証データに
for filenum in range(30):
  shutil.move('drive/MyDrive/Colab Notebooks/dataset/inuneko/train/dog/' +str(filenum+1).zfill(6)+'.jpg', 'drive/MyDrive/Colab Notebooks/dataset/inuneko/test/dog')

In [10]:
# 学習に必要なものをimport
import torch
import torch.nn as nn
import torch.optim as optim
import torch.onnx #Burracudaで使う用
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
from tqdm import tqdm
from PIL import Image

In [11]:
# 画像の前処理
size = (224, 224)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_data_dir = 'drive/MyDrive/Colab Notebooks/dataset/inuneko/train'
val_data_dir = 'drive/MyDrive/Colab Notebooks/dataset/inuneko/test'

image_datasets = {
    'val': torchvision.datasets.ImageFolder(val_data_dir, transform=data_transforms['val']),
    'train': torchvision.datasets.ImageFolder(train_data_dir, transform=data_transforms['train'])
}

In [12]:
# 学習データを取り出しやすくした形
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=10, shuffle=True),
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=1)
}

In [13]:
# 画像の枚数やラベルを定義
dataset_sizes = {
    'train': len(image_datasets['train']),
    'val': len(image_datasets['val'])
}

class_names = image_datasets['train'].classes
print('分類種類:', class_names)

分類種類: ['cat', 'dog']


In [14]:
# 転移学習用のモデルのダウンロードと設定

model = models.vgg16(pretrained=True)

# パラメータの固定
for param in model.parameters():
    param.requires_grad = False

# 最後の全結合層を固定しない＞ここだけ学習する
last_layer = list(model.children())[-1]
for param in last_layer.parameters():
    param.requires_grad = True

# 学習時にGPUを使えるなら使う
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Burracudaで学習させたい場合はCPUを使う
device = torch.device("cpu")


# 分類数を1000から2つに変更
num_ftrs = model.classifier[6].in_features
model.classifier[6] = torch.nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [15]:
# loss関数の定義

# lossを定義
criterion = nn.CrossEntropyLoss()

# 色々な最適化関数 lrが学習率 0.001 0.0001などで調整
optimizer = optim.Adam(model.parameters(), lr=0.0001,)
# optimizer = optim.SGD(model.parameters(), lr=0.001,)

In [17]:
# 学習部分の定義

def train(model, dataloader, otpimizer, criterion, num_epochs, device):
    """
    model:学習モデル
    dataloader:学習、評価データのdataloader
    optimizer:最適化関数
    crierion:ロス関数
    num_epochs:学習回数
    device:CPUかGPUか
    """
    best_acc = 0.0
    # 学習を繰り返す
    for epoch in range(num_epochs):
        # trainとvalを繰り返す
        for phase in ['train', 'val']:
            # モデルを学習モードか評価モードに切り替える
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            # 精度計算用
            loss_sum = 0.0
            acc_sum = 0.0
            total = 0

            # 進捗の表示
            with tqdm(total=len(dataloaders[phase]),unit="batch") as pbar:
                pbar.set_description(f"Epoch[{epoch}/{num_epochs}]({phase})")
                
                # dataloadersからバッチサイズに応じてデータを取得
                for inputs, labels in dataloaders[phase]:
                    # 画像とラベルをGPU/CPUか切り替え
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # 予測
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    
                    # ロス算出
                    loss = criterion(outputs, labels)

                    # 予測とラベルの差を使って学習 
                    if phase == 'train':
                        # ここは決まり文句
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                    # ロス、精度を算出
                    total += inputs.size(0)
                    loss_sum += loss.item() * inputs.size(0)
                    acc_sum += torch.sum(preds == labels.data).item()
                    # 進捗の表示
                    pbar.set_postfix({"loss":loss_sum/float(total),"accuracy":float(acc_sum)/float(total)})
                    pbar.update(1)

            # 1エポックでのロス、精度を算出
            epoch_loss = loss_sum / dataset_sizes[phase]
            epoch_acc = acc_sum / dataset_sizes[phase]

            # 一番良い制度の時にモデルデータを保存
            if phase == 'val' and epoch_acc > best_acc:
                print(f"save model epoch:{epoch} loss:{epoch_loss} acc:{epoch_acc}")

                dummy_input = torch.randn(1, 3, 224, 224)
                torch.onnx.export(model, dummy_input,'best_model.onnx') # Burracudaで使うようの形式で出力
                torch.save(model, 'best_model.pth')

In [ ]:
# 学習開始

num_epochs = 10
train(model, dataloaders, optimizer, criterion, num_epochs, device)

Epoch[0/10](train):   0%|          | 0/14 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Epoch[0/10](val): 100%|██████████| 60/60 [00:56<00:00,  1.07batch/s, loss=0.131, accuracy=0.933]


save model epoch:0 loss:0.13077660098541108 acc:0.9333333333333333


Epoch[1/10](val): 100%|██████████| 60/60 [00:37<00:00,  1.61batch/s, loss=0.00162, accuracy=1]


save model epoch:1 loss:0.0016232235678252493 acc:1.0


Epoch[2/10](val):  78%|███████▊  | 47/60 [00:29<00:07,  1.66batch/s, loss=0.000371, accuracy=1]

In [ ]:
# 学習モデルの使用

best_model = torch.load('best_model.pth') # 学習モデルの読み込み

# 対象画像
filepath = "drive/MyDrive/Colab Notebooks/images/"
filename = 'dog02'
filetype = ".jpg" 

# 読み込み画像をリサイズやtensorなどの方に変換
input_image = Image.open(filepath + filename + filetype)
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

# GPU使える場合はGPUを使う
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    best_model.to('cuda')

# AIの判定
with torch.no_grad():
    output = best_model(input_batch)
output = torch.nn.functional.softmax(output[0], dim=0)
print(output.shape)

# 出力結果から2種類のうちどれかを数値で取得
output = output.to('cpu').detach().numpy().copy()
ind = np.argmax(output)
print(class_names[ind])